In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import gc
gc.collect()

0

In [2]:
flow_with_sqrt=pd.read_csv('../data/flow_df_with_sqrt.csv')
flow_no_sqrt=pd.read_csv('../data/flow_df_not_sqrt.csv')
print(flow_with_sqrt.shape,flow_no_sqrt.shape)
print(flow_with_sqrt.columns,flow_no_sqrt.columns)
tmp=flow_no_sqrt[['date_dt','address_str','start_address','end_address','flow_out_not_sqrt','flow_in_not_sqrt']]
flow=flow_with_sqrt.merge(tmp,on=['date_dt','address_str','start_address','end_address'],how='left')

(28322, 15) (28322, 15)
Index(['address', 'address_str', 'city_code', 'date_dt', 'day',
       'district_code', 'dwell', 'flow_in', 'flow_out', 'month', 'weekd',
       'start_address', 'flow_out_sqrt', 'end_address', 'flow_in_sqrt'],
      dtype='object') Index(['address', 'address_str', 'city_code', 'date_dt', 'day',
       'district_code', 'dwell', 'flow_in', 'flow_out', 'month', 'weekd',
       'start_address', 'flow_out_not_sqrt', 'end_address',
       'flow_in_not_sqrt'],
      dtype='object')


In [3]:
f_col=['flow_in','flow_out','flow_out_sqrt', 'flow_in_sqrt', 'flow_out_not_sqrt','flow_in_not_sqrt']
for fea in f_col:
    flow[fea]=flow[fea].apply(lambda x:round(x,6))

In [4]:
flow['in_bigger_out']=np.where(flow['flow_in']>flow['flow_out'],1,0)

In [5]:
flow['in_bigger_out_nosqrt']=np.where(flow['flow_in_not_sqrt']>flow['flow_out_not_sqrt'],1,0)

In [6]:
flow['in_bigger_out_sqrt']=np.where(flow['flow_in_sqrt']>flow['flow_out_sqrt'],1,0)

In [7]:
flow['should_same']=0
flow.loc[(flow['in_bigger_out']==flow['in_bigger_out_nosqrt'])&(flow['in_bigger_out']==flow['in_bigger_out_sqrt']),'should_same']=2
flow.loc[(flow['in_bigger_out']==flow['in_bigger_out_nosqrt'])|(flow['in_bigger_out']==flow['in_bigger_out_sqrt']),'should_same']=1

In [8]:
gc.collect()

48

In [9]:
pair_col=[('flow_out_sqrt', 'flow_in_sqrt'), 
          ('flow_out_not_sqrt','flow_in_not_sqrt')]
for i in range(2):
    lst=pair_col[i]
    fea_name=str(i)+'_mean'
    flow[fea_name]=round((flow[lst[0]]+flow[lst[1]])/2,6)
    
for i in range(2):
    lst=pair_col[i]
    fea_name=str(i)+'_sub'
    flow[fea_name]=round((flow[lst[0]]/flow[lst[1]]),6)
    
for i in range(2):
    lst=pair_col[i]
    fea_name=str(i)+'_del'
    flow[fea_name]=round((flow[lst[0]]-flow[lst[1]]),6)

In [10]:
print(flow.shape) #(28322, ?)

print(len(flow[flow['dwell'].isnull()])) #1470

(28322, 27)
1470


In [11]:
#把已有的比例赋值给没有的
true_col=['dwell','flow_in','flow_out']
for i in range(3):
    for j in range(i+1,3):
        name1=true_col[i]+'_'+true_col[j]
        flow[name1]=round(flow[true_col[i]]/flow[true_col[j]],6)
        
        name2=true_col[i]+'_mean_'+true_col[j]
        flow[name2]=round((flow[true_col[i]]+flow[true_col[j]])/2,6)

print(flow.shape) #(28322, ?)
print(flow[flow['dwell'].isnull()].shape) #1470,?

(28322, 33)
(1470, 33)


In [12]:
flow.to_csv('../data/flowV50.csv',index=False)

In [119]:
to_col=['dwell_flow_in', 'dwell_mean_flow_in','dwell_flow_out', 'dwell_mean_flow_out', 'flow_in_flow_out',
       'flow_in_mean_flow_out']

for fea in to_col:
    train=flow[flow['dwell'].notnull()][['address','day','weekd',fea]]
    tmp=train.groupby(['address','day','weekd'])[fea].mean().reset_index(name=fea+'_n')
    flow=flow.merge(tmp,on=['address','day','weekd'],how='left')
flow=flow.drop(to_col,axis=1)
del tmp
print(flow.shape) #(28322, ?)
print(len(flow[flow['dwell'].isnull()])) #1470
gc.collect()

(28322, 31)
1470


251

In [120]:
# #这个评估标准是大赛规定的
def rmsle(y_true, y_pred):
    return 'RMSLE', round(np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))),6), False

In [121]:
address=list(set(flow['address_str']))

In [122]:
train=flow[flow['dwell'].notnull()]
print(train.shape,train.columns)
test=flow[flow['dwell'].isnull()]
print(test.shape,test.columns)

(26852, 31) Index(['address', 'address_str', 'city_code', 'date_dt', 'day',
       'district_code', 'dwell', 'flow_in', 'flow_out', 'month', 'weekd',
       'flow_out_sqrt', 'flow_in_sqrt', 'flow_out_not_sqrt',
       'flow_in_not_sqrt', 'in_bigger_out', 'in_bigger_out_nosqrt',
       'in_bigger_out_sqrt', 'all_big', '0_mean', '1_mean', '0_sub', '1_sub',
       '0_del', '1_del', 'dwell_flow_in_n', 'dwell_mean_flow_in_n',
       'dwell_flow_out_n', 'dwell_mean_flow_out_n', 'flow_in_flow_out_n',
       'flow_in_mean_flow_out_n'],
      dtype='object')
(1470, 31) Index(['address', 'address_str', 'city_code', 'date_dt', 'day',
       'district_code', 'dwell', 'flow_in', 'flow_out', 'month', 'weekd',
       'flow_out_sqrt', 'flow_in_sqrt', 'flow_out_not_sqrt',
       'flow_in_not_sqrt', 'in_bigger_out', 'in_bigger_out_nosqrt',
       'in_bigger_out_sqrt', 'all_big', '0_mean', '1_mean', '0_sub', '1_sub',
       '0_del', '1_del', 'dwell_flow_in_n', 'dwell_mean_flow_in_n',
       'dwell_flow_o

In [123]:
feature=['address','day','weekd', 'flow_out_sqrt', 'flow_in_sqrt','flow_out_not_sqrt', 
         'flow_in_not_sqrt', 'in_bigger_out',
       'in_bigger_out_nosqrt', 'in_bigger_out_sqrt', 'all_big', '0_mean',
       '1_mean', '0_sub', '1_sub', '0_del', '1_del', 'dwell_flow_in_n',
       'dwell_mean_flow_in_n', 'dwell_flow_out_n', 'dwell_mean_flow_out_n',
       'flow_in_flow_out_n', 'flow_in_mean_flow_out_n']

label=['flow_in','flow_out','dwell']

In [124]:
traindf=train[train['date_dt']<20180210]
valdf=train[train['date_dt']>=20180210]
print(traindf.shape,valdf.shape)

(24892, 31) (1960, 31)


In [125]:
result = pd.DataFrame(columns=['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in', 'flow_out'])

In [126]:
import lightgbm as lgb
from lightgbm import LGBMRegressor

In [ ]:
#有98个ad了哦
for ad in address:
    for y in label:
        #已知训练集
        train_x= traindf[traindf['address_str']==ad][feature]
        train_y = traindf[traindf['address_str']==ad][y]
        ##已知验证集
        val_x = valdf[valdf['address_str']==ad][feature]
        val_y = valdf[valdf['address_str']==ad][y]
        
        testdf=test[test['address_str']==ad]
        
        #由于每一个ad对应的只有274个数据，所以不用再分了，直接验证rmsle并预测
        gbm = lgb.LGBMRegressor(boosting_type='gbdt',
                            objective='regression',
                            num_leaves=70,
                            subsample=0.9, colsample_bytree=0.9, 
                            learning_rate=0.05,
                            n_estimators=1000)

        gbm.fit(train_x, train_y, 
                eval_set=[(train_x,train_y),(val_x, val_y)],
                eval_metric='rmse',
                early_stopping_rounds=100)

        y_pred = gbm.predict(val_x, num_iteration=gbm.best_iteration_)
        print('The rmsle on val is:', rmsle(val_y, y_pred)[1])
        print("====================================================================")

        gc.collect()
        
        testdf[y] = gbm.predict(testdf[feature],num_iteration=gbm.best_iteration_)
        
        result = pd.concat([result,testdf[result.columns]],axis=0)
print(result.shape,result.columns)

[1]	training's rmse: 154.318	valid_1's rmse: 74.2458
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 148.198	valid_1's rmse: 74.2049
[3]	training's rmse: 142.421	valid_1's rmse: 74.2702
[4]	training's rmse: 137.063	valid_1's rmse: 75.1196
[5]	training's rmse: 131.987	valid_1's rmse: 75.4064
[6]	training's rmse: 127.199	valid_1's rmse: 75.7818
[7]	training's rmse: 122.723	valid_1's rmse: 75.9627
[8]	training's rmse: 118.581	valid_1's rmse: 76.9654
[9]	training's rmse: 114.618	valid_1's rmse: 77.5244
[10]	training's rmse: 110.948	valid_1's rmse: 78.1017
[11]	training's rmse: 107.404	valid_1's rmse: 78.8892
[12]	training's rmse: 104.129	valid_1's rmse: 79.6249
[13]	training's rmse: 101.14	valid_1's rmse: 80.0559
[14]	training's rmse: 98.2514	valid_1's rmse: 80.85
[15]	training's rmse: 95.607	valid_1's rmse: 81.7249
[16]	training's rmse: 93.1143	valid_1's rmse: 82.3043
[17]	training's rmse: 90.7997	valid_1's rmse: 82.6594
[18]	training's rmse: 88.6288	va

[1]	training's rmse: 64.4852	valid_1's rmse: 40.9078
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 62.1748	valid_1's rmse: 39.6911
[3]	training's rmse: 60.1014	valid_1's rmse: 38.5222
[4]	training's rmse: 58.0256	valid_1's rmse: 37.4139
[5]	training's rmse: 56.0519	valid_1's rmse: 36.2927
[6]	training's rmse: 54.2318	valid_1's rmse: 35.186
[7]	training's rmse: 52.5639	valid_1's rmse: 34.2114
[8]	training's rmse: 50.9892	valid_1's rmse: 33.3087
[9]	training's rmse: 49.4978	valid_1's rmse: 32.5754
[10]	training's rmse: 48.033	valid_1's rmse: 31.7041
[11]	training's rmse: 46.6615	valid_1's rmse: 30.8818
[12]	training's rmse: 45.3663	valid_1's rmse: 30.2886
[13]	training's rmse: 44.1643	valid_1's rmse: 29.6742
[14]	training's rmse: 43.0573	valid_1's rmse: 28.9871
[15]	training's rmse: 42.0091	valid_1's rmse: 28.3983
[16]	training's rmse: 41.0426	valid_1's rmse: 27.9804
[17]	training's rmse: 40.1157	valid_1's rmse: 27.4372
[18]	training's rmse: 39.268	v

[1]	training's rmse: 142.654	valid_1's rmse: 48.4485
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 137.15	valid_1's rmse: 48.1259
[3]	training's rmse: 131.895	valid_1's rmse: 47.8272
[4]	training's rmse: 127.045	valid_1's rmse: 47.8989
[5]	training's rmse: 122.371	valid_1's rmse: 48.0857
[6]	training's rmse: 117.982	valid_1's rmse: 48.459
[7]	training's rmse: 113.771	valid_1's rmse: 48.8914
[8]	training's rmse: 109.897	valid_1's rmse: 49.1686
[9]	training's rmse: 106.244	valid_1's rmse: 49.8545
[10]	training's rmse: 102.743	valid_1's rmse: 50.6637
[11]	training's rmse: 99.4328	valid_1's rmse: 51.258
[12]	training's rmse: 96.4663	valid_1's rmse: 51.5009
[13]	training's rmse: 93.6497	valid_1's rmse: 52.3291
[14]	training's rmse: 90.9423	valid_1's rmse: 53.1684
[15]	training's rmse: 88.4903	valid_1's rmse: 53.5026
[16]	training's rmse: 86.2324	valid_1's rmse: 54.3218
[17]	training's rmse: 83.7271	valid_1's rmse: 55.034
[18]	training's rmse: 81.7324	va

[1]	training's rmse: 9.08567	valid_1's rmse: 7.39916
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 8.6786	valid_1's rmse: 6.99768
[3]	training's rmse: 8.30455	valid_1's rmse: 6.62829
[4]	training's rmse: 7.94336	valid_1's rmse: 6.29948
[5]	training's rmse: 7.59945	valid_1's rmse: 6.00822
[6]	training's rmse: 7.27473	valid_1's rmse: 5.75236
[7]	training's rmse: 6.9708	valid_1's rmse: 5.53324
[8]	training's rmse: 6.68221	valid_1's rmse: 5.34681
[9]	training's rmse: 6.41264	valid_1's rmse: 5.19193
[10]	training's rmse: 6.15593	valid_1's rmse: 5.06576
[11]	training's rmse: 5.91638	valid_1's rmse: 4.96752
[12]	training's rmse: 5.69085	valid_1's rmse: 4.88282
[13]	training's rmse: 5.47671	valid_1's rmse: 4.84202
[14]	training's rmse: 5.27624	valid_1's rmse: 4.81574
[15]	training's rmse: 5.08619	valid_1's rmse: 4.80472
[16]	training's rmse: 4.90958	valid_1's rmse: 4.81721
[17]	training's rmse: 4.7427	valid_1's rmse: 4.84029
[18]	training's rmse: 4.58769	v

[1]	training's rmse: 12.9522	valid_1's rmse: 11.3831
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 12.4078	valid_1's rmse: 10.7019
[3]	training's rmse: 11.896	valid_1's rmse: 10.0735
[4]	training's rmse: 11.4113	valid_1's rmse: 9.48903
[5]	training's rmse: 10.9561	valid_1's rmse: 8.94211
[6]	training's rmse: 10.5254	valid_1's rmse: 8.45567
[7]	training's rmse: 10.1162	valid_1's rmse: 7.98084
[8]	training's rmse: 9.73657	valid_1's rmse: 7.54134
[9]	training's rmse: 9.37369	valid_1's rmse: 7.14472
[10]	training's rmse: 9.03412	valid_1's rmse: 6.77067
[11]	training's rmse: 8.71455	valid_1's rmse: 6.46173
[12]	training's rmse: 8.41331	valid_1's rmse: 6.16914
[13]	training's rmse: 8.13104	valid_1's rmse: 5.90575
[14]	training's rmse: 7.8654	valid_1's rmse: 5.79535
[15]	training's rmse: 7.61581	valid_1's rmse: 5.59637
[16]	training's rmse: 7.38117	valid_1's rmse: 5.53541
[17]	training's rmse: 7.16378	valid_1's rmse: 5.49173
[18]	training's rmse: 6.95113	

...

[1]	training's rmse: 28.5267	valid_1's rmse: 29.5673
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 27.3491	valid_1's rmse: 29.8353
[3]	training's rmse: 26.2452	valid_1's rmse: 30.101
[4]	training's rmse: 25.1957	valid_1's rmse: 30.2895
[5]	training's rmse: 24.2117	valid_1's rmse: 30.4769
[6]	training's rmse: 23.2879	valid_1's rmse: 30.7145
[7]	training's rmse: 22.4258	valid_1's rmse: 30.9452
[8]	training's rmse: 21.6067	valid_1's rmse: 31.1106
[9]	training's rmse: 20.8438	valid_1's rmse: 31.8076
[10]	training's rmse: 20.1293	valid_1's rmse: 32.847
[11]	training's rmse: 19.4563	valid_1's rmse: 32.9781
[12]	training's rmse: 18.8429	valid_1's rmse: 33.8713
[13]	training's rmse: 18.2545	valid_1's rmse: 34.0575
[14]	training's rmse: 17.7089	valid_1's rmse: 34.9315
[15]	training's rmse: 17.191	valid_1's rmse: 35.0277
[16]	training's rmse: 16.7103	valid_1's rmse: 35.1532
[17]	training's rmse: 16.2427	valid_1's rmse: 35.06
[18]	training's rmse: 15.8317	val

[119]	training's rmse: 14.2538	valid_1's rmse: 29.5489
[120]	training's rmse: 14.218	valid_1's rmse: 29.5856
[121]	training's rmse: 14.1883	valid_1's rmse: 29.5827
[122]	training's rmse: 14.163	valid_1's rmse: 29.6007
[123]	training's rmse: 14.1372	valid_1's rmse: 29.652
Early stopping, best iteration is:
[23]	training's rmse: 23.0152	valid_1's rmse: 23.3544
The rmsle on val is: 0.154008
[1]	training's rmse: 47.8583	valid_1's rmse: 35.6313
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 45.8292	valid_1's rmse: 34.2133
[3]	training's rmse: 43.912	valid_1's rmse: 32.9112
[4]	training's rmse: 42.1051	valid_1's rmse: 31.7527
[5]	training's rmse: 40.3985	valid_1's rmse: 30.6337
[6]	training's rmse: 38.7914	valid_1's rmse: 29.6337
[7]	training's rmse: 37.2829	valid_1's rmse: 28.8219
[8]	training's rmse: 35.8564	valid_1's rmse: 28.0227
[9]	training's rmse: 34.5206	valid_1's rmse: 27.3548
[10]	training's rmse: 33.2681	valid_1's rmse: 26.7575
[11]	training's 

[1]	training's rmse: 272.778	valid_1's rmse: 346.513
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 261.015	valid_1's rmse: 340.259
[3]	training's rmse: 249.944	valid_1's rmse: 336.026
[4]	training's rmse: 239.432	valid_1's rmse: 332.907
[5]	training's rmse: 229.56	valid_1's rmse: 328.497
[6]	training's rmse: 220.202	valid_1's rmse: 325.606
[7]	training's rmse: 211.484	valid_1's rmse: 322.312
[8]	training's rmse: 203.203	valid_1's rmse: 318.929
[9]	training's rmse: 195.412	valid_1's rmse: 316.236
[10]	training's rmse: 188.121	valid_1's rmse: 313.341
[11]	training's rmse: 181.237	valid_1's rmse: 312.643
[12]	training's rmse: 174.767	valid_1's rmse: 311.14
[13]	training's rmse: 168.742	valid_1's rmse: 308.733
[14]	training's rmse: 163.086	valid_1's rmse: 307.568
[15]	training's rmse: 157.751	valid_1's rmse: 306.653
[16]	training's rmse: 152.778	valid_1's rmse: 305.696
[17]	training's rmse: 148.165	valid_1's rmse: 304.325
[18]	training's rmse: 143.706	

[124]	training's rmse: 75.8392	valid_1's rmse: 296.898
[125]	training's rmse: 75.7113	valid_1's rmse: 297.363
[126]	training's rmse: 75.4527	valid_1's rmse: 297.318
[127]	training's rmse: 75.2066	valid_1's rmse: 297.112
[128]	training's rmse: 74.9515	valid_1's rmse: 297.262
Early stopping, best iteration is:
[28]	training's rmse: 116.863	valid_1's rmse: 264.769
The rmsle on val is: 0.13192
[1]	training's rmse: 79.356	valid_1's rmse: 69.1521
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 76.1199	valid_1's rmse: 67.8524
[3]	training's rmse: 73.0701	valid_1's rmse: 66.9586
[4]	training's rmse: 70.1632	valid_1's rmse: 65.921
[5]	training's rmse: 67.4711	valid_1's rmse: 64.5042
[6]	training's rmse: 64.8918	valid_1's rmse: 63.6452
[7]	training's rmse: 62.4385	valid_1's rmse: 63.2261
[8]	training's rmse: 60.1958	valid_1's rmse: 62.2053
[9]	training's rmse: 57.9769	valid_1's rmse: 62.224
[10]	training's rmse: 55.8948	valid_1's rmse: 62.2967
[11]	training's 

[1]	training's rmse: 89.7476	valid_1's rmse: 41.4152
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 86.2511	valid_1's rmse: 42.269
[3]	training's rmse: 82.962	valid_1's rmse: 43.17
[4]	training's rmse: 79.8754	valid_1's rmse: 44.1158
[5]	training's rmse: 76.9314	valid_1's rmse: 45.1109
[6]	training's rmse: 74.2055	valid_1's rmse: 46.081
[7]	training's rmse: 71.6418	valid_1's rmse: 47.129
[8]	training's rmse: 69.2214	valid_1's rmse: 48.101
[9]	training's rmse: 66.9987	valid_1's rmse: 49.0232
[10]	training's rmse: 64.907	valid_1's rmse: 49.9322
[11]	training's rmse: 62.9768	valid_1's rmse: 50.8213
[12]	training's rmse: 61.1247	valid_1's rmse: 51.6784
[13]	training's rmse: 59.4153	valid_1's rmse: 52.9599
[14]	training's rmse: 57.7807	valid_1's rmse: 53.6912
[15]	training's rmse: 56.3248	valid_1's rmse: 54.3758
[16]	training's rmse: 54.9666	valid_1's rmse: 55.537
[17]	training's rmse: 53.7004	valid_1's rmse: 56.1886
[18]	training's rmse: 52.4341	valid_1

In [103]:
# result['date_dt'] = result['date_dt'].astype(int)
# result = result[['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in', 'flow_out']]
# print(pd.isnull(result).sum())
# print(result.shape)
# gc.collect()

date_dt          0
city_code        0
district_code    0
dwell            0
flow_in          0
flow_out         0
dtype: int64
(144060, 6)


24

In [104]:
# result.to_csv('..\data\predictionV5.csv', index=False, header=None)